Roboflow가 Training, Validation 셋 shuffle이나 stratify 안하고 업로드한 순서대로만 스플릿 해서 일일이 테스트 데이터 셋 만드는 code

- 우선 클래스 별로 이미지 검색
- 해당 클래스 별 이미지 장수에 따라 내가 알아서 적절히 테스트 셋 이미지 장수 정함
- 이미지 검색 기능 이용해서 웹 스크래핑 이용해 내가 delete 후에 validation 스플릿으로 다시 업로드할 이미지들의 파일명을 따로 저장하기 위한 작업


00_strawberry(정상): 600장


In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
import os
import json

In [ ]:
#A1

#처음부터 다 다시 올릴거임

사유 universe.roboflow 랑 app.roboflow 보여지는 이미지들 다름


In [ ]:
imgs_dir = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_딸기/딸기_병해/병해'
a_imgs = os.listdir(imgs_dir)

In [ ]:
len(a_imgs)

3014

In [ ]:
a1_imgs = []

In [ ]:
categories_list = ['a1_strawberry']

In [ ]:
idx = 0
ibm_data = {
    "version": "1.0",
    "type": "localization",
    "labels": categories_list,
    "annotations":{}
}

annot_dir = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL2_딸기/병해'
labels_list = os.listdir(annot_dir)
json_files = [jfile for jfile in labels_list if jfile.endswith('.json')]
for json_file in json_files:
  file_path = os.path.join(annot_dir, json_file)
  with open(file_path, 'r') as file:
    data = json.load(file)
    if data["description"]["image"] in a_imgs:
      if data['annotations']['disease'] == 'a1':
        a1_imgs.append(data["description"]["image"])
        image_filename = data["description"]["image"]
        bbox = data["annotations"]["bbox"][0]
        bbox_x = int(bbox["x"])
        bbox_y = int(bbox["y"])
        bbox_w = int(bbox["w"])
        bbox_h = int(bbox["h"])
        bbox_x2 = bbox_x + bbox_w
        bbox_y2 = bbox_y + bbox_h
        class_name = f"{data['annotations']['disease']}_strawberry"
        class_name = class_name.replace('딸기','strawberry')

        value = [{
            "x":bbox_x,
            "y":bbox_y,
            "x2":bbox_x2,
            "y2":bbox_y2,
            "id":str(idx),
            "label":class_name
        }]
        idx += 1
        ibm_data["annotations"][image_filename] = value

output_directory = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL2_딸기/annotations/랜덤 학습이미지들 (1차)'
output_file_path = os.path.join(output_directory,'strawberry_a1_reupload'+".json")
with open(output_file_path,"w") as output_json:
  json.dump(ibm_data,output_json,ensure_ascii=False)

In [ ]:
len(a1_imgs)

1748

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="2nxKEUjSzlIMWDl3hWnV")
print(rf.workspace())
project = rf.workspace().project("strawberry-rbo1c")

In [ ]:
for img in a1_imgs:
  img_path = os.path.join(imgs_dir,img)
  project.upload(img_path,output_file_path)

In [ ]:
'''url_list = [
    'https://universe.roboflow.com/capstone-qhqkt/strawberry-rbo1c/browse?queryText=class%3Aa1_strawberry&pageSize=50&startingIndex=0&browseQuery=true',
    'https://universe.roboflow.com/capstone-qhqkt/strawberry-rbo1c/browse?queryText=class%3Aa1_strawberry&pageSize=50&startingIndex=400&browseQuery=true',
    'https://universe.roboflow.com/capstone-qhqkt/strawberry-rbo1c/browse?queryText=class%3Aa1_strawberry&pageSize=50&startingIndex=800&browseQuery=true',
    'https://universe.roboflow.com/capstone-qhqkt/strawberry-rbo1c/browse?queryText=class%3Aa1_strawberry&pageSize=50&startingIndex=1200&browseQuery=true',
    'https://universe.roboflow.com/capstone-qhqkt/strawberry-rbo1c/browse?queryText=class%3Aa2_strawberry&pageSize=20&startingIndex=0&browseQuery=true',
    'https://universe.roboflow.com/capstone-qhqkt/strawberry-rbo1c/browse?queryText=class%3Aa2_strawberry&pageSize=20&startingIndex=220&browseQuery=true',
    'https://universe.roboflow.com/capstone-qhqkt/strawberry-rbo1c/browse?queryText=class%3Aa2_strawberry&pageSize=20&startingIndex=440&browseQuery=true',
    'https://universe.roboflow.com/capstone-qhqkt/strawberry-rbo1c/browse?queryText=class%3Aa2_strawberry&pageSize=20&startingIndex=660&browseQuery=true'
]'''

#파일들 읽어서 처리

In [ ]:
imgs = []
files = '/content/delete_c1.txt'
with open(files,'r') as file:
  content = file.read()
  strings = content
  soup = BeautifulSoup(strings,'html.parser')
  paragraphs = soup.find_all('p')
  for p in paragraphs:
    imgs.append(p.text)

In [ ]:
imgs = imgs[:30]

In [ ]:
len(imgs)

30

In [ ]:
files = [
    '/content/delete_c2_1.txt',
    '/content/delete_c2_2.txt',
    '/content/delete_c2_3.txt'
]
strings = []
for f in files:
  with open(f,'r') as file:
    content = file.read()
    strings.append(content)

In [ ]:
for string in strings:
  soup = BeautifulSoup(string,'html.parser')
  paragraphs = soup.find_all('p')
  for p in paragraphs:
    imgs.append(p.text)

In [ ]:
img1_set = set(imgs1)
img2_set = set(imgs2)

In [ ]:
print(len(img1_set))
print(len(img2_set))

60
40


In [ ]:
z = img1_set.difference(img2_set)

In [ ]:
len(imgs)

90

In [ ]:
annot_file_path = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL2_딸기/annotations/랜덤 학습이미지들 (1차)/strawberry_for_validation_c12.json'
imgs_dir = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_딸기/딸기_작물보호제처리반응/작물보호제처리반응'
for img in imgs:
  img_path = os.path.join(imgs_dir,img)
  project.upload(img_path,annot_file_path,
                 split='valid')


#한번에 하나씩

In [ ]:
'''with open('/content/delete_a2_4.txt','r') as file:
  content = file.read()
  strings = content'''

In [ ]:
imgs = []
for string in strings:
  soup = BeautifulSoup(string,'html.parser')
  paragraphs = soup.find_all('p')
  for p in paragraphs:
    imgs.append(p.text)

In [ ]:
len(imgs)

60

In [ ]:
imgs

In [ ]:
annot_file_path = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL2_딸기/annotations/랜덤 학습이미지들 (1차)/strawberry_for_validation_a2_1.json'
imgs_dir = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_딸기/딸기_병해/병해'
for img in imgs:
  img_path = os.path.join(imgs_dir,img)
  project.upload(img_path,annot_file_path,
                 split='valid')


#여러개한꺼번에

In [ ]:
categories_list = ['c1_strawberry','c2_strawberry']

In [ ]:
idx = 0
ibm_data = {
    "version": "1.0",
    "type": "localization",
    "labels": categories_list,
    "annotations":{}
}

annot_dir = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL2_딸기/작물보호제처리반응'
for img in imgs:
  filename = img.replace('jpg','').replace('JPG','')
  annot_path = os.path.join(annot_dir,filename)+'json'
  with open(annot_path) as json_file:
    data = json.load(json_file)
  if data["description"]["image"] == img:
    image_filename = data["description"]["image"]
    bbox = data["annotations"]["bbox"][0]
    bbox_x = int(bbox["x"])
    bbox_y = int(bbox["y"])
    bbox_w = int(bbox["w"])
    bbox_h = int(bbox["h"])
    bbox_x2 = bbox_x + bbox_w
    bbox_y2 = bbox_y + bbox_h
    class_name = f"{data['annotations']['disease']}_strawberry"
    class_name = class_name.replace('딸기','strawberry')

    value = [{
        "x":bbox_x,
        "y":bbox_y,
        "x2":bbox_x2,
        "y2":bbox_y2,
        "id":str(idx),
        "label":class_name
    }]
    idx += 1
    ibm_data["annotations"][image_filename] = value


output_directory = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL2_딸기/annotations/랜덤 학습이미지들 (1차)'
output_file_path = os.path.join(output_directory,'strawberry_for_validation_c12'+".json")
with open(output_file_path,"w") as output_json:
  json.dump(ibm_data,output_json,ensure_ascii=False)

In [ ]:
!pip install Roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="2nxKEUjSzlIMWDl3hWnV")
print(rf.workspace())
project = rf.workspace().project("strawberry-rbo1c")

In [ ]:
annot_file_path = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL2_딸기/annotations/랜덤 학습이미지들 (1차)/strawberry_for_validation_normal.json'
imgs_dir = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_딸기/딸기_정상/정상'
for img in imgs:
  img_path = os.path.join(imgs_dir,img)
  project.upload(img_path,annot_file_path,
                 split='valid')